In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os

In [2]:
os.chdir('/Users/Sriram/Desktop/DePaul/Work - LIDC (Smart CAD)/LIDC')

In [3]:
data = np.loadtxt('./labels.txt', delimiter='\t', skiprows=1, dtype='int64')

In [4]:
#Choosing a single label based on mode, in case of tie, choose the first occurence
newLabels = np.zeros(shape=[data.shape[0]])
for i in range(data.shape[0]):
    bins = np.bincount(data[i,:])
    newLabels[i] = np.argmax(bins)

IndexError: too many indices for array

In [6]:
data.shape

(809,)

In [7]:
df = pd.read_csv('./LIDC_REU2015.csv', index_col=0)
df_sub = df.iloc[:,10:75]

In [9]:
df_sub

,Area,ConvexArea,Perimeter,ConvexPerimeter,EquivDiameter,MajorAxisLength,MinorAxisLength,Elongation,Compactness,Eccentricity,...,Correlation,Energy,Homogeneity,Entropy,x_3rdordermoment,Inversevariance,Sumaverage,Variance,Clustertendency,MaxProbability
InstanceID,,,,,,,,,,,,,,,,,,,,,
4,1094,1286,168.852814,135.372918,37.321898,42.803687,35.834900,1.194469,2.073908,0.546909,...,0.894,0.0003,0.0532,8.15,1290000,0.0223,661.0,143000,541000,0.0009
134,931,1062,148.267027,119.799290,34.429435,39.635819,30.844618,1.285016,1.879012,0.628016,...,0.899,0.0004,0.0572,8.03,735000,0.0247,416.0,111000,420000,0.0010
143,161,167,51.455844,49.201081,14.317527,18.125068,11.574663,1.565926,1.308681,0.769539,...,0.692,0.0027,0.0312,6.03,-469000,0.0107,420.0,98700,333000,0.0043
118,246,251,58.769553,57.721132,17.697936,18.324991,17.321312,1.057945,1.117274,0.326408,...,0.793,0.0018,0.0764,6.43,8600000,0.0392,565.0,137000,492000,0.0052
175,71,73,32.142136,31.494737,9.507892,10.528352,8.908660,1.181811,1.157927,0.532931,...,0.634,0.0076,0.0296,4.98,15500000,0.0097,403.0,200000,650000,0.0105
179,67,67,30.142136,29.963990,9.236182,10.025928,8.619837,1.163123,1.079104,0.510707,...,0.489,0.0085,0.0315,4.86,1160000,0.0105,221.0,17300,51300,0.0095
195,120,127,42.627417,41.063967,12.360774,13.946049,11.230660,1.241784,1.205000,0.592877,...,0.641,0.0038,0.0358,5.69,2830000,0.0143,320.0,59200,194000,0.0053
297,72,80,35.213203,33.228232,9.574615,12.504386,7.807575,1.601571,1.370467,0.781115,...,0.600,0.0062,0.0247,5.19,6800000,0.0069,350.0,50300,161000,0.0071
318,89,95,37.455844,36.698532,10.645108,13.213481,8.992606,1.469372,1.254409,0.732690,...,0.574,0.0046,0.0345,5.48,5910000,0.0072,309.0,95700,300000,0.0076


In [16]:
minmax = lambda x: (x-x.min())/(x.max()-x.min())
df_norm = df_sub.apply(minmax, 0)

In [17]:
df_norm.to_csv("./LIDCfeatureData.csv", header=False, index=False)

In [18]:
newLabels = pd.DataFrame(data)
newLabels.to_csv('./LIDCLabels.csv', header=False, index=False)

In [19]:
from sklearn.cross_validation import train_test_split

In [20]:
train, test, train_Labels, test_Labels = train_test_split(df_norm, newLabels, test_size = 0.33)
train.to_csv("./LIDCtrain.csv", header=False, index=False)
test.to_csv("./LIDCtest.csv", header=False, index=False)
train_Labels.to_csv("./LIDCtrianLabels.csv", header=False, index=False)
test_Labels.to_csv("./LIDCtestLabels.csv", header=False, index=False)

In [21]:
def SDMat(trainlabels = train_Labels):
    labels = train_Labels.values.flatten()
    global S; global D
    S = np.zeros(shape=[labels.shape[0],labels.shape[0]])
    D = np.ones(shape=[labels.shape[0],labels.shape[0]])
    for i in range(len(labels)):
        for j in range(i,len(labels)):
            if labels[i] == labels[j]:
                S[i,j] = 1; S[j,i] = 1
                D[i,j] = 0; D[j,i] = 0
    S = pd.DataFrame(S); D = pd.DataFrame(D)

In [22]:
SDMat()
S.to_csv('./LIDCSMatrix.csv', header=False, index=False); D.to_csv('./LIDCDMatrix.csv', header=False, index=False)

In [24]:
from sklearn import neighbors

predLabels5 = np.loadtxt('./5nn.csv', delimiter=',')
nn = 5
kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions5 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [25]:
test_Labels.values.flatten()

array([2, 2, 5, 2, 3, 2, 3, 2, 3, 3, 5, 3, 4, 3, 5, 5, 3, 4, 1, 3, 3, 2, 3,
       2, 2, 1, 5, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 2, 1, 3, 4, 3, 3, 1,
       3, 1, 4, 1, 2, 4, 3, 3, 4, 3, 5, 2, 3, 5, 3, 3, 3, 1, 5, 3, 4, 5, 2,
       3, 5, 3, 4, 2, 2, 4, 3, 2, 1, 3, 3, 1, 4, 3, 4, 1, 3, 2, 3, 1, 3, 4,
       2, 1, 5, 2, 3, 3, 2, 3, 1, 1, 5, 1, 5, 3, 3, 1, 3, 3, 5, 4, 4, 3, 4,
       3, 5, 1, 2, 4, 3, 4, 3, 2, 2, 3, 4, 4, 3, 3, 1, 3, 4, 1, 2, 4, 3, 2,
       3, 3, 3, 1, 3, 2, 3, 1, 3, 5, 3, 1, 2, 4, 3, 4, 5, 3, 3, 2, 3, 2, 2,
       3, 3, 2, 2, 4, 1, 1, 4, 3, 5, 3, 3, 5, 5, 2, 3, 3, 4, 3, 2, 3, 3, 4,
       2, 3, 3, 4, 5, 2, 4, 4, 5, 4, 2, 2, 1, 3, 4, 5, 3, 5, 3, 3, 3, 3, 4,
       5, 2, 3, 1, 2, 3, 3, 3, 3, 2, 2, 1, 3, 3, 3, 1, 4, 5, 3, 3, 3, 4, 3,
       1, 4, 3, 3, 2, 2, 3, 2, 3, 3, 2, 2, 1, 1, 1, 3, 5, 4, 3, 3, 2, 5, 4,
       3, 3, 4, 3, 4, 4, 2, 2, 3, 2, 1, 3, 3, 3])

In [36]:
nn = 5

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions5 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [38]:
sum(predictions5.flatten() == test_Labels.values.flatten())/len(predictions5)

0.29213483146067415

In [41]:
sum(predLabels5.flatten() == test_Labels.values.flatten())/len(predLabels5)

0.27715355805243447

In [182]:
predLabels3 = np.loadtxt('./predLabels3.csv', delimiter=','); predLabels.shape
nn = 3

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [177]:
sum(predictions3.flatten() == test_Labels.flatten())/len(predLabels)

0.43445692883895132

In [184]:
sum(predLabels3.flatten() == test_Labels.flatten())/len(predLabels)

0.43820224719101125

In [188]:
predLabels7 = np.loadtxt('./predLabels7.csv', delimiter=','); predLabels.shape
nn = 7

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions7 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [190]:
sum(predictions7.flatten() == test_Labels.flatten())/len(predLabels)

0.50187265917602997

In [189]:
sum(predLabels7.flatten() == test_Labels.flatten())/len(predLabels)

0.4943820224719101

In [220]:
predLabels3 = np.loadtxt('./predLabelsAll3.csv', delimiter=','); predLabels.shape
nn = 3

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [217]:
sum(predictions3 == test_Labels.values.flatten())/len(predLabels3)

0.4044943820224719

In [221]:
sum(predLabels3 == test_Labels.values.flatten())/len(predLabels3)

0.38951310861423222

In [222]:
predLabels5 = np.loadtxt('./predLabelsAll5.csv', delimiter=','); predLabels.shape
nn = 5

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [223]:
sum(predictions5 == test_Labels.values.flatten())/len(predLabels5)

0.43445692883895132

In [226]:
sum(predLabels5 == test_Labels.values.flatten())/len(predLabels5)

0.44569288389513106

In [231]:
predLabels7 = np.loadtxt('./predLabelsAll7.csv', delimiter=','); predLabels.shape
nn = 7

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions7 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [229]:
sum(predictions7 == test_Labels.values.flatten())/len(predLabels5)

0.49063670411985016

In [232]:
sum(predLabels7 == test_Labels.values.flatten())/len(predLabels7)

0.47565543071161048